In [1]:
# Step 1: Install the necessary libraries
!pip install gradio nltk scikit-learn --quiet

In [ ]:
# Step 2: Import the required libraries
import gradio as gr
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import time

In [ ]:
# Step 3: Download required NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords', quiet=True)

nltk.download('punkt_tab')

In [ ]:
# Define the FAQ Knowledge Base
faqs = [
    { "question": "What is WordPress?", "answer": "WordPress is a free and open-source content management system (CMS). It's a popular tool for individuals and businesses to create websites and blogs." },
    { "question": "Difference between WordPress.com and WordPress.org?", "answer": "WordPress.org is the self-hosted version where you have full control. WordPress.com is a hosted service that is easier to start with but offers less flexibility." },
    { "question": "How do I install WordPress?", "answer": "Most web hosts offer a 'one-click' installation. You can also install it manually by downloading it from WordPress.org and uploading it to your web server." },
    { "question": "What are plugins?", "answer": "Plugins are add-ons you can install to add new features to your website, like contact forms, e-commerce stores, or photo galleries." },
    { "question": "What are themes?", "answer": "Themes control the visual design and layout of your website. You can easily change your theme to give your site a new look." },
    { "question": "How do I add a new post or page?", "answer": "From your dashboard, go to 'Posts' -> 'Add New' for a blog post, or 'Pages' -> 'Add New' for a static page." },
    { "question": "Is WordPress secure?", "answer": "The core software is very secure. Security risks usually come from outdated plugins or themes. Always keep everything updated!" },
    { "question": "Can I build an e-commerce store?", "answer": "Yes, absolutely. Use popular plugins like WooCommerce to turn your WordPress site into a fully functional online store." }
]

In [ ]:
# Define the Text Preprocessing Function
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return " ".join(filtered_tokens)

In [ ]:
# --- One-time setup for the NLP model ---
print("Setting up the chatbot model...")
all_questions = [faq["question"] for faq in faqs]
processed_questions = [preprocess_text(q) for q in all_questions]
vectorizer = TfidfVectorizer()
question_vectors = vectorizer.fit_transform(processed_questions)
print("Chatbot is ready!")

In [ ]:
# Define the function that finds the best answer
def get_best_answer(user_message):
    processed_message = preprocess_text(user_message)
    if not processed_message:
        return "Please ask a more specific question."

    message_vector = vectorizer.transform([processed_message])
    similarities = cosine_similarity(message_vector, question_vectors)[0]
    most_similar_index = np.argmax(similarities)

    if similarities[most_similar_index] > 0.2: # Confidence threshold
        return faqs[most_similar_index]["answer"]
    else:
        return "I'm sorry, I don't have a specific answer for that. Could you please try rephrasing your question?"

In [ ]:
# This function will handle the chat interaction in the UI
def chat_function(message, history):
    # Get the response from our simple backend
    bot_response = get_best_answer(message)

    # Simulate typing for a better user experience
    response_with_typing = ""
    for char in bot_response:
        response_with_typing += char
        time.sleep(0.02)
        # Update the history with the gradually typed response
        history[-1] = (message, response_with_typing)
        yield history

# This function handles clearing the chat
def clear_chat_function():
    return []


In [ ]:
# --- Event Handlers ---

    # Function to add the user message to the chat and get the bot's response
    def respond(message, chat_history):
        # Add the user's message to the chat history
        chat_history.append((message, None))
        # Use a generator to stream the bot's response
        for history_update in chat_function(message, chat_history):
            yield history_update

    # Connect the textbox submission to the respond function
    txt_input.submit(
        fn=respond,
        inputs=[txt_input, chatbot_window],
        outputs=[chatbot_window]
    ).then(
        # Clear the textbox after submission
        lambda: gr.update(value=""), None, [txt_input], queue=False
    )

    # Connect the clear button to its function
    clear_button.click(clear_chat_function, [], [chatbot_window])

    # Add examples for users to easily click and try
    gr.Examples(
        examples=[
            "What are plugins for?",
            "How do I make an online store?",
            "How safe is WordPress?",
            "What is the difference between a post and a page?"
        ],
        inputs=txt_input,
        label="Click an example to ask:"
    )

# Launch the Gradio application
if __name__ == "__main__":
    demo.launch(debug=True, share=True)